#### Lab 5 - Análisis de tráfico, implementación de un IDS

Parte 1

In [1]:
from scapy.config import conf
conf.use_pcap = True

ModuleNotFoundError: No module named 'scapy'

In [2]:
pip install --pre scapy[basic]

zsh:1: no matches found: scapy[basic]
Note: you may need to restart the kernel to use updated packages.
